# (Shared/Local) Memory and Barriers in Reductions

**NB:** Equivalent names:

* CUDA: Shared Memory
* OpenCL: Local Memory
* Computer Architecture: Scratchpad

In [2]:
import pyopencl as cl
import pyopencl.reduction
import pyopencl.tools

In [3]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

Choose platform:
[0] <pyopencl.Platform 'NVIDIA CUDA' at 0x22a10a0>
[1] <pyopencl.Platform 'Intel(R) OpenCL' at 0x21f9d88>
[2] <pyopencl.Platform 'Portable Computing Language' at 0x7f679cfba5c0>
Choice [0]:
Choose device(s):
[0] <pyopencl.Device 'GeForce GTX TITAN X' on 'NVIDIA CUDA' at 0x22ad0a0>
[1] <pyopencl.Device 'GeForce GTX TITAN X' on 'NVIDIA CUDA' at 0x22b0f80>
Choice, comma-separated [0]:
Set the environment variable PYOPENCL_CTX=':' to avoid being asked again.


In [13]:
code = cl.reduction._get_reduction_source(
    ctx,
    out_type="float", out_type_size=4,
    neutral="0", reduce_expr="a+b", map_expr="x[i]",
    parsed_args=cl.tools.parse_arg_list("float *x"))
src = code.source

# {{{ clear double empty lines

import re
src, _ = re.subn(r"\n\s+\n", "\n\n", src)

while True:
    oldsrc = src
    src = src.replace("\n\n\n", "\n\n")
    if oldsrc == src:
        break
        
# }}}
        
print(src)

//CL//
    #define PCL_GROUP_SIZE 256
    #define PCL_READ_AND_MAP(i) (x[i])
    #define PCL_REDUCE(a, b) (a+b)

        #if __OPENCL_C_VERSION__ < 120
        #pragma OPENCL EXTENSION cl_khr_fp64: enable
        #endif
        #define PYOPENCL_DEFINE_CDOUBLE

    #include <pyopencl-complex.h>

    typedef float pcl_out_type;

    __kernel void reduce_kernel(
      __global pcl_out_type *pcl_out__base, long pcl_out__offset,
      __global float *x, 
      long pcl_start, long pcl_step, long pcl_stop,
      unsigned int pcl_seq_count, long n)
    {
        __global pcl_out_type *pcl_out = (__global pcl_out_type *) (
            (__global char *) pcl_out__base + pcl_out__offset);

        __local pcl_out_type pcl_ldata[PCL_GROUP_SIZE];

        unsigned int pcl_lid = get_local_id(0);

        const long pcl_base_idx =
            get_group_id(0)*PCL_GROUP_SIZE*pcl_seq_count + pcl_lid;
        long i = pcl_start + pcl_base_idx * pcl_step;

        pcl_out_type pcl_acc = 0;
        for (un

In [14]:
prg = cl.Program(ctx, src).build()